In [1]:
import pandas as pd
import numpy as np
import config
import datetime
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
dico={}

In [3]:
def get_ps(year, specialite):
    sql=f"""
select sp.specialite_id, s.label_long as label, ds.annee as year, ps.id as p_id, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join date_source ds on ds.id=tds.date_source_id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
where sp.specialite_id={specialite}
and ds.annee={year}
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)


In [4]:
def get_pa(year, specialite):
    sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and ds.annee={year}
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [5]:
def get_pa_ps(year, specialite):
    sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, coalesce(an.lon,an2.lon) as lon, coalesce(an.lat,an2.lat) as lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join date_source ds on ds.id=pands.date_source_id
left outer join ps on ps.key=pa.inpp
join tarif t on t.ps_id=ps.id
join tarif_date_source tds on tds.tarif_id=t.id and tds.date_source_id=ds.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an2 on ar.adresse_norm_id=an2.id
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and ds.annee={year}
"""
    print(f"Quering PA-PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [6]:
def get_rpps(year, specialite):
    sql=f"""
select personne.id p_id,an.lon,an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
where sp.specialite_id={specialite}
and personne.date_maj >= '20{year}-01-01' and date_effet <= '20{year}-12-31'
"""
    print(f"Quering RPPS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [7]:
df2=get_ps(year,specialite)
df2

Quering PS for year 21 and specialite 10


,specialite_id,label,year,p_id,lon,lat
0,10,Généraliste,21,36253,-0.237617,44.943571
1,10,Généraliste,21,36253,-0.237617,44.943571
2,10,Généraliste,21,36253,-0.237617,44.943571
3,10,Généraliste,21,36253,-0.237617,44.943571
4,10,Généraliste,21,36253,-0.237617,44.943571
...,...,...,...,...,...,...
2849256,10,Généraliste,21,36097,-0.628736,44.856396
2849257,10,Généraliste,21,36097,-0.628736,44.856396
2849258,10,Généraliste,21,36097,-0.628736,44.856396
2849259,10,Généraliste,21,36097,-0.628736,44.856396


In [8]:
nb_unique = len(df2.groupby(["p_id", "lon", "lat"]))
nb_unique

58241

In [9]:
nb = df2["p_id"].nunique()
nb

54851

In [10]:
ameli={10:51488}
ameli

{10: 51488}

In [24]:
df = pd.DataFrame(columns=["specialite","label","year","ameli","nb_ps_cabinet","nb_ps","nb_pa_cabinet","nb_pa","nb_pa_ps_cabinet","nb_pa_ps","nb_rpps_cabinet","np_rpps"])
for specialite in range(1,11):
    for yy in range(20,datetime.date.today().year+1-2000):
        ps_df=get_ps(yy,specialite)
        label=ps_df["label"][0] if len(ps_df["label"])>0 else ""
        nb_unique_ps=len(ps_df.groupby(["p_id", "lon", "lat"]))
        nb_ps=ps_df["p_id"].nunique()
        pa_df=get_pa(yy,specialite)
        nb_unique_pa=len(pa_df.groupby(["p_id", "lon", "lat"]))
        nb_pa=pa_df["p_id"].nunique()
        pa_ps_df=get_pa_ps(yy,specialite)
        pa_ps_df=pa_ps_df.dropna()
        nb_unique_pa_ps=len(pa_ps_df.groupby(["p_id", "lon", "lat"]))
        nb_pa_ps=pa_ps_df["p_id"].nunique()
        rpps_df=get_rpps(yy,specialite)
        rpps_df=rpps_df.dropna()
        nb_unique_rpps=len(rpps_df.groupby(["p_id", "lon", "lat"]))
        nb_rpps=rpps_df["p_id"].nunique()
        a=np.nan
        if yy==23 and specialite in ameli:
            a=ameli[specialite]
        df = pd.concat([pd.DataFrame([[specialite,label,yy,a,nb_unique_ps,nb_ps,nb_unique_pa,nb_pa,nb_unique_pa_ps,nb_pa_ps,nb_unique_rpps,nb_rpps]], columns=df.columns), df], ignore_index=True)
df=df.sort_values(by=['specialite',"year"])
df

Quering PS for year 20 and specialite 1
Quering PA for year 20 and specialite 1
Quering PA-PS for year 20 and specialite 1
Quering RPPS for year 20 and specialite 1
Quering PS for year 21 and specialite 1
Quering PA for year 21 and specialite 1
Quering PA-PS for year 21 and specialite 1
Quering RPPS for year 21 and specialite 1
Quering PS for year 22 and specialite 1
Quering PA for year 22 and specialite 1
Quering PA-PS for year 22 and specialite 1
Quering RPPS for year 22 and specialite 1
Quering PS for year 23 and specialite 1
Quering PA for year 23 and specialite 1
Quering PA-PS for year 23 and specialite 1
Quering RPPS for year 23 and specialite 1
Quering PS for year 24 and specialite 1
Quering PA for year 24 and specialite 1
Quering PA-PS for year 24 and specialite 1
Quering RPPS for year 24 and specialite 1
Quering PS for year 25 and specialite 1
Quering PA for year 25 and specialite 1
Quering PA-PS for year 25 and specialite 1
Quering RPPS for year 25 and specialite 1
Quering PS

,specialite,label,year,ameli,nb_ps_cabinet,nb_ps,nb_pa_cabinet,nb_pa,nb_pa_ps_cabinet,nb_pa_ps,nb_rpps_cabinet,np_rpps
59,1,Psychiatre,20,NaN,5915,5646,0,0,0,0,6779,6779
58,1,Psychiatre,21,NaN,6319,5829,6586,5232,5114,4226,6854,6854
57,1,Psychiatre,22,NaN,5287,5062,6480,5159,0,0,6900,6900
56,1,,23,NaN,0,0,0,0,0,0,6935,6935
55,1,,24,NaN,0,0,0,0,0,0,6966,6966
54,1,,25,NaN,0,0,6541,4928,0,0,84,84
53,2,Anesthésiste,20,NaN,3952,3434,0,0,0,0,1205,1205
52,2,Anesthésiste,21,NaN,4194,3599,5191,3641,4518,3032,1206,1206
51,2,Anesthésiste,22,NaN,3788,3310,5306,3768,0,0,1205,1205
50,2,,23,NaN,0,0,0,0,0,0,1207,1207


In [26]:
df.to_excel("ps_stats.xlsx", index=False)